In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/24 14:59:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_spark = spark.read.csv('test1.csv',header = True, inferSchema = True )
df_spark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:

df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
df_spark.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','Experience'],outputCol= "Independent_features")

In [8]:
output = featureassembler.transform(df_spark)


In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent_features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
assembler3 = VectorAssembler(inputCols=['age','Experience','Salary'], outputCol='ind3')

In [12]:
output3= assembler3.transform(df_spark)
output3.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|               ind3|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|[31.0,10.0,30000.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0,25000.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0,20000.0]|
|     Paul| 24|         3| 20000| [24.0,3.0,20000.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0,15000.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0,18000.0]|
+---------+---+----------+------+-------------------+



In [14]:

output.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Independent_features: vector (nullable = true)



In [15]:
finalized_data = output.select('Independent_features','Salary')
finalized_data.show()

+--------------------+------+
|Independent_features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [27]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor= LinearRegression(featuresCol= 'Independent_features',labelCol='Salary')
regressor=regressor.fit(train_data)


22/11/24 15:44:39 WARN Instrumentation: [6b81bf00] regParam is zero, which might cause numerical instability and overfitting.


In [28]:
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [29]:
regressor.intercept

13333.333333333807

In [30]:
test_data.show()

+--------------------+------+
|Independent_features|Salary|
+--------------------+------+
|          [23.0,2.0]| 18000|
|          [24.0,3.0]| 20000|
|          [30.0,8.0]| 25000|
+--------------------+------+



In [31]:
pred_results = regressor.evaluate(train_data)

In [32]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent_features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|15000.000000000051|
|          [29.0,4.0]| 20000| 19999.99999999994|
|         [31.0,10.0]| 30000|30000.000000000015|
+--------------------+------+------------------+



/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [33]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(4.123042648037275e-11, 2.0646428089323575e-21)